# 1. Setup Driver

In [2]:
import sys
sys.path.append("../")
from src.driver import setup_driver
driver = setup_driver()

# 2. Access to KINEO

In [4]:
from src.login import login
from private.config import USER_KEY, PASS_KEY

# login KINEO
login(driver,
      url = 'https://aforadores.mitma.es/contadorestraficofomento/Login.aspx',
      username = USER_KEY, 
      password = PASS_KEY)

In [5]:
# from src.dropdown import get_all_combinations
# driver.get('https://aforadores.mitma.es/contadorestraficofomento/InformeVolumenTraficoAgrupadoAforo.aspx')
# get_all_combinations(driver, 'ctl00_ContentPlaceHolderDatos_CbDemarcacion_B-1', 'ctl00_ContentPlaceHolderDatos_CbEtd_B-1', '../../input/values.py')

# 3. Make the requests and download data

In [7]:
from src.download import download_data
from input.values import VALUES_gerard_prova

for values in VALUES_gerard_prova:
    download_data(driver,
                  demarcacion = values["demarcacion"],
                  etd = values["etd"],
                  fecha_inicio = '01/06/2022',
                  fecha_fin = '02/06/2022',
                  option = 'por_minutos' # por_horas
                 )

Exit driver

In [9]:
driver.quit()

# 4. Move files to desired directory

In [11]:
from src.directory import move_to_directory
from private.config import DOWNLOADS_PATH, L_PATH, G_PATH

move_to_directory(origin = DOWNLOADS_PATH,
                  destination = G_PATH)

NameError: name 'G_PATH' is not defined